In [ ]:
import pandas as pd
import time

# Assuming single rate for given aq, not incremental (for different usage)
# Limits on starting date based on first data point in rate table
start_time = time.time()
""""""""""""""""
CHANGE THE PATH TO WHERE YOUR EXCEL IS LOCATED
"""""""""""""""""

file = pd.ExcelFile(r'/Users/juliankautsch/PycharmProjects/GorillaAssesment/gorilla_test_data.xlsx')
rate_table = pd.read_excel(file, 'rate_table')
meter_list = pd.read_excel(file, 'meter_list')
forecast_table = pd.read_excel(file, 'forecast_table')
meters_list = []

# this loop could be enclosed in function and run  in parallel with multiprocessing.Process
# change in how the data is stored after the loop would be required
for i in range(len(meter_list)):
    meterid = meter_list['meter_id'][i]
    aq = meter_list['aq_kwh'][i]
    ExitZone = meter_list['exit_zone'][i]
    rates_1 = rate_table[(rate_table.exit_zone == ExitZone) & (rate_table.aq_min_kwh <= aq) &
                         ((rate_table.aq_max_kwh >= aq) | (rate_table.aq_max_kwh.isnull()))]
    rate_dates = rates_1['date']
    rate = pd.DataFrame(rates_1['rate_p_per_kwh'])
    forecast_1 = forecast_table[forecast_table.meter_id == meterid]
    estimated_cost = forecast_1['kwh']
    forecast_dates = forecast_1['date']
    consumption = 0
    totalCost = 0
    n = 0
    a = 0
    ConsumptionList = []
    # find starting rate - there aren't too many time periods, so quick solution
    # general case solution, could be calculated only once if all starting dates are the same
    # if this was the case also Consumption calculation could be divided into same size chunks
    while forecast_dates.iloc[0] > rate_dates.iloc[n]:
        if forecast_dates.iloc[0] < rate_dates.iloc[n + 1]: break
        n += 1
    starting_n = n

    # for starting conditions
    consumption_in_period = 0
    for i in rate_dates:
        if n < len(rate_dates) - 1:
            lastConsumption = consumption_in_period
            consumption_in_period = forecast_1.loc[(forecast_1['date'] >= rate_dates.iloc[n]) &
                                                   (forecast_1['date'] < rate_dates.iloc[n + 1]), "kwh"].sum()
        else:
            consumption_in_period = forecast_1.loc[(forecast_1['date'] >= rate_dates.iloc[n]), "kwh"].sum()
        n += 1
        ConsumptionList.append(consumption_in_period)
        if (consumption_in_period == 0) & (lastConsumption > 0) | n == len(rate_dates): break

    consumption_df = pd.DataFrame(ConsumptionList, columns=['kwh'])
    total_cost = round((consumption_df.values * rate.iloc[starting_n:n].values / 100).sum(), 2)
    total_consumption = estimated_cost.sum()
    meter_info = [meterid, total_consumption, total_cost]
    meters_list.append(meter_info)
df = pd.DataFrame(meters_list, columns=['Meter ID', 'Total Estimated Consumption (kWh)', 'Total Cost (£)'])
print(df)
print("--- %s seconds ---" % (time.time() - start_time))